In [ ]:
# this code has been adapted from code created by Andrew Ross - https://github.com/andrew-c-ross/nwa-shared
from glob import glob
from subprocess import run
from os import path
import xarray
from boundary import Segment

# xarray gives a lot of unnecessary warnings
import warnings
warnings.filterwarnings('ignore')


def write_year(year, glorys_dir, segments, is_first_year=False):
    glorys = (
        xarray.open_mfdataset(sorted(glob(path.join(glorys_dir, f'{year}/GLORYS_REANALYSIS_{year}-*.nc'))))
        .rename({'latitude': 'lat', 'longitude': 'lon', 'depth': 'z'})
    )

    # Floor first time down to midnight so that it matches initial conditions
    if is_first_year:
       tnew = xarray.concat((glorys['time'][0].dt.floor('1d'), glorys['time'][1:]), dim='time')
       glorys['time'] = ('time', tnew)

    for seg in segments:
        seg.regrid_velocity(glorys['uo'], glorys['vo'], suffix=year, flood=False)
        for tr in ['thetao', 'so']:
            seg.regrid_tracer(glorys[tr], suffix=year, flood=False)
        seg.regrid_tracer(glorys['zos'], suffix=year, flood=False)


def ncrcat_years(nsegments, output_dir):            
    for var in ['thetao', 'so', 'zos', 'uv']:
        for seg in range(1, nsegments+1):
            run([f'ncrcat -O {var}_{seg:03d}_* {var}_{seg:03d}.nc'], cwd=output_dir, shell=True)


def main():
    first_year = 1995
    glorys_dir = '/Volumes/P1/Data/GLORYS/'
    output_dir = '/Users/james/Documents/glorys_obc_gen/'
    hgrid = xarray.open_dataset('/Users/james/Documents/nwa25/ocean_hgrid.nc')
    segments = [
        Segment(1, 'south', hgrid, output_dir=output_dir),
        Segment(2, 'north', hgrid, output_dir=output_dir),
        Segment(3, 'east', hgrid, output_dir=output_dir)
    ]
 
    for y in range(1995, 1996):
        print(y)
        write_year(y, glorys_dir, segments, is_first_year=y==first_year)
    
    ncrcat_years(len(segments), output_dir)


if __name__ == '__main__':
    main()